# Mini Datathon F5 #

En este análisis, exploraremos las métricas clave de mortalidad, hospitalizaciones, casos positivos y vacunados por estados en Estados Unidos. Estos datos nos ayudarán a comprender el impacto de la pandemia, evaluar la eficacia de las medidas tomadas y monitorear el progreso de la vacunación en el país.


In [16]:
# IMPORTAMOS BIBLIOTECAS

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

# CARGA DE DATOS

# URL de la API
url = "https://api.covidtracking.com/v1/states/daily.json"

# Realizar la solicitud (request) para obtener los datos JSON
response = requests.get(url)

# Verificar que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Cargar los datos JSON en un DataFrame de pandas
    data = response.json()
    df = pd.DataFrame(data)
    
    # Mostrar el DataFrame
    display(df)
else:
    print("No se pudieron obtener los datos JSON. Código de estado:", response.status_code)




,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,None,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,None,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,None,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,None,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,None,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20775,20200117,WA,0.0,NaN,NaN,NaN,totalTestEncountersViral,NaN,NaN,NaN,...,None,0,0,7cefac6b3681020741ca30f45399a7b22f2e45b4,0,0,0,0,0,
20776,20200116,WA,0.0,NaN,NaN,NaN,totalTestEncountersViral,NaN,NaN,NaN,...,None,0,0,650501e005a5ee86d93c5f32dda56735ea2af967,0,0,0,0,0,
20777,20200115,WA,0.0,NaN,NaN,NaN,totalTestEncountersViral,NaN,NaN,NaN,...,None,0,0,4987e61aad88182abfe641033b597304c2153d4f,0,0,0,0,0,
20778,20200114,WA,0.0,NaN,NaN,NaN,totalTestEncountersViral,NaN,NaN,NaN,...,None,0,0,1881c8a2f0d337b22066b4f05df06eb2259e8d57,0,0,0,0,0,


## Análisis exploratorio preliminar

Dataset con **20780** entradas y **56 campos** de los cuales 14 están marcados como "deprecated" (a desestimar) en la documentación del proveedor de datos  https://covidtracking.com/data/api --> "Historic values for all states ".

En el caso de campos con muchos datos incompletos, se valorará el método para lidiar con ellos (eliminación filas, columnas, imputación etc) en caso necesario. El campo **dataQualityGrade** puede ser eliminado porque no contiene ningún valor.


In [17]:
# Definimos las columnas que queremos eliminar
columnas_eliminar = ['checkTimeEt', 'commercialScore', 'dateChecked', 'dateModified', 'grade', 'hash', 
                     'hospitalized', 'negativeIncrease', 'negativeRegularScore', 'negativeScore', 
                     'posNeg', 'positiveScore', 'score', 'total', 'dataQualityGrade']

# Creamos un nuevo dataframe que no incluya esas columnas (deprecated)
df1 = df.drop(columns=columnas_eliminar)

# Información sobre el dataset (tipo de datos, campos nulos o vacíos)

df1.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         20780 non-null  int64  
 1   state                        20780 non-null  object 
 2   positive                     20592 non-null  float64
 3   probableCases                9271 non-null   float64
 4   negative                     13290 non-null  float64
 5   pending                      2138 non-null   float64
 6   totalTestResultsSource       20780 non-null  object 
 7   totalTestResults             20614 non-null  float64
 8   hospitalizedCurrently        17339 non-null  float64
 9   hospitalizedCumulative       12382 non-null  float64
 10  inIcuCurrently               11636 non-null  float64
 11  inIcuCumulative              3789 non-null   float64
 12  onVentilatorCurrently        9126 non-null   float64
 13  onVentilatorCumu

In [18]:
# asignamos datetime a la variable date (actualmente float)
df1['date'] = pd.to_datetime(df1['date'], format='%Y%m%d')
df1.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,negativeTestsPeopleAntibody,totalTestsPeopleAntigen,positiveTestsPeopleAntigen,totalTestsAntigen,positiveTestsAntigen,fips,positiveIncrease,totalTestResultsIncrease,deathIncrease,hospitalizedIncrease
0,2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,NaN,NaN,NaN,NaN,02,0,0,0,0
1,2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,NaN,NaN,NaN,NaN,01,408,2347,-1,0
2,2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,481311.0,81803.0,NaN,NaN,05,165,3380,22,11
3,2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,60,0,0,0,0
4,2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,NaN,NaN,NaN,NaN,04,1335,45110,5,44


In [19]:
# Análisis estadístico básico
df1.describe()

,date,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,positiveTestsPeopleAntibody,negativeTestsPeopleAntibody,totalTestsPeopleAntigen,positiveTestsPeopleAntigen,totalTestsAntigen,positiveTestsAntigen,positiveIncrease,totalTestResultsIncrease,deathIncrease,hospitalizedIncrease
count,20780,2.059200e+04,9271.000000,1.329000e+04,2138.000000,2.061400e+04,17339.000000,12382.000000,11636.000000,3789.000000,...,1094.000000,972.000000,999.000000,633.000000,3.421000e+03,2233.000000,20780.000000,20780.000000,20780.000000,20780.000000
mean,2020-09-02 18:13:30.779595776,1.651560e+05,21729.123719,8.482246e+05,1659.862956,2.186936e+06,1190.576965,9262.762478,359.621176,1934.191607,...,20516.966179,188710.969136,168188.246246,25259.048973,3.089196e+05,31837.241379,1383.849519,17508.388210,24.790712,37.360780
min,2020-01-13 00:00:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.000000,0.000000,6.000000,...,0.000000,1.000000,3.000000,3.000000,1.000000e+00,0.000000,-7757.000000,-130545.000000,-201.000000,-12257.000000
25%,2020-06-02 00:00:00,5.753750e+03,773.500000,5.394125e+04,43.000000,1.040498e+05,166.500000,985.250000,60.000000,501.000000,...,3155.500000,54874.000000,37675.500000,2682.000000,2.004700e+04,1085.000000,65.000000,1205.750000,0.000000,0.000000
50%,2020-09-03 00:00:00,4.606450e+04,4491.000000,3.059720e+05,212.500000,6.552670e+05,531.000000,4472.000000,172.000000,1295.000000,...,11956.000000,100282.000000,144130.000000,17763.000000,1.233840e+05,13661.000000,435.000000,6125.000000,6.000000,0.000000
75%,2020-12-05 00:00:00,1.779580e+05,19553.000000,1.056611e+06,1295.750000,2.264766e+06,1279.000000,12248.500000,380.000000,2451.000000,...,19059.000000,261121.000000,255251.000000,47012.000000,4.327270e+05,49010.000000,1335.250000,19086.500000,24.000000,36.000000
max,2021-03-07 00:00:00,3.501394e+06,365961.000000,1.018694e+07,64400.000000,4.964601e+07,22851.000000,82237.000000,5225.000000,9263.000000,...,178979.000000,816231.000000,580372.000000,81803.000000,2.664340e+06,211546.000000,71734.000000,473076.000000,2559.000000,16373.000000
std,NaN,3.267852e+05,45471.459778,1.344501e+06,4671.028548,4.436508e+06,2060.041207,12620.544081,594.831150,1953.329983,...,29267.559775,200218.073746,143748.073365,24139.253458,4.232854e+05,41929.658070,3023.558742,33586.022461,60.162742,208.237151
